# Fourier series in numpy

## code snippets from stackoverflow for refrence

In [4]:
def fourier_series_coeff_numpy(f, T, N, return_complex=False):
    """Calculates the first 2*N+1 Fourier series coeff. of a periodic function.

    Given a periodic, function f(t) with period T, this function returns the
    coefficients a0, {a1,a2,...},{b1,b2,...} such that:

    f(t) ~= a0/2+ sum_{k=1}^{N} ( a_k*cos(2*pi*k*t/T) + b_k*sin(2*pi*k*t/T) )

    If return_complex is set to True, it returns instead the coefficients
    {c0,c1,c2,...}
    such that:

    f(t) ~= sum_{k=-N}^{N} c_k * exp(i*2*pi*k*t/T)

    where we define c_{-n} = complex_conjugate(c_{n})

    Refer to wikipedia for the relation between the real-valued and complex
    valued coeffs at http://en.wikipedia.org/wiki/Fourier_series.

    Parameters
    ----------
    f : the periodic function, a callable like f(t)
    T : the period of the function f, so that f(0)==f(T)
    N_max : the function will return the first N_max + 1 Fourier coeff.

    Returns
    -------
    if return_complex == False, the function returns:

    a0 : float
    a,b : numpy float arrays describing respectively the cosine and sine coeff.

    if return_complex == True, the function returns:

    c : numpy 1-dimensional complex-valued array of size N+1

    """
    # From Shanon theoreom we must use a sampling freq. larger than the maximum
    # frequency you want to catch in the signal.
    f_sample = 2 * N
    # we also need to use an integer sampling frequency, or the
    # points will not be equispaced between 0 and 1. We then add +2 to f_sample
    t, dt = np.linspace(0, T, f_sample + 2, endpoint=False, retstep=True)

    y = np.fft.rfft(f(t)) / t.size

    if return_complex:
        return y
    else:
        y *= 2
        return y[0].real, y[1:-1].real, -y[1:-1].imag


In [6]:
from numpy import ones_like, cos, pi, sin, allclose
T = 1.5  # any real number

def f(t):
    """example of periodic function in [0,T]"""
    n1, n2, n3 = 1., 4., 7.  # in Hz, or nondimensional for the matter.
    a0, a1, b4, a7 = 4., 2., -1., -3
    return a0 / 2 * ones_like(t) + a1 * cos(2 * pi * n1 * t / T) + b4 * sin(
        2 * pi * n2 * t / T) + a7 * cos(2 * pi * n3 * t / T)


N_chosen = 10
a0, a, b = fourier_series_coeff_numpy(f, T, N_chosen)

# we have as expected that
assert allclose(a0, 4)
assert allclose(a, [2, 0, 0, 0, 0, 0, -3, 0, 0, 0])
assert allclose(b, [0, 0, 0, -1, 0, 0, 0, 0, 0, 0])

In [2]:
import numpy as np

def main():
    # Create the x and y arrays
    x = np.array([0, 30, 60, 90, 120, 150])
    y = np.array(list(map(float, input().split())))

    # Get the number of harmonics
    harmonic = int(input("How many harmonics do you require for your analysis: "))

    # Create the table of harmonic coefficients
    table = generate_harmonic_analysis_table(x, y, harmonic)

    # Print the table
    print_table(table)

    # Print the summations of the columns
    print_summations(table)

def r2d(num):
    return num * np.pi / 180

def generate_harmonic_analysis_table(x, y, harmonic=3):
    """
    Generates a table of harmonic coefficients for the given x and y data.

    Args:
        x: An array of x values.
        y: An array of y values.
        harmonic: The number of harmonics to include.

    Returns:
        A table of harmonic coefficients.
    """

    table = np.zeros((len(x), harmonic*2+1))

    for r in range(len(x)):
        # Add the y value to the table
        table[r][0] = y[r]

        # Add the cosine and sine coefficients to the table
        for i in range(harmonic):
            table[r][i+1] = y[r] * np.cos(r2d((i+1)*x[r]))
        for i in range(harmonic):
            table[r][i+1+harmonic] = y[r] * np.sin(r2d((i+1)*x[r]))

    return table

def print_table(table):
    """
    Prints the given table.

    Args:
        table: The table to print.
    """

    print("X\t\tY\t\t", end="")

    for i in range(table.shape[1]-2):
        print("Ycosx"+str(i+1)+"\t\t", end="")

    print("Ysinx"+str(i+1)+"\t\t")

    for row in table:
        print("\n", end="")
        for value in row:
            print(round(value, 2), end="\t\t")

def print_summations(table):
    """
    Prints the summations of the columns in the given table.

    Args:
        table: The table to print the summations for.
    """

    print("\n\nHere are the summations of the columns of the table, curated for you :")
    print("Y       = ", table[:, 0].sum())

    print("Ycosx   = ", table[:, 1].sum())
    for i in range(table.shape[1]-3):
        print("Ycos"+str(i+2)+"x  = ", table[:, i+2].sum())

    print("Ysinx   = ", table[:, table.shape[1]-2].sum())
    for i in range(table.shape[1]-3):
        print("Ysin"+str(i+2)+"x  = ", table[:, i+3].sum())

if __name__ == "__main__":
    main()


 1 2 3 4 5 6 
How many harmonics do you require for your analysis:  2


X		Y		Ycosx1		Ycosx2		Ycosx3		Ysinx3		

1.0		1.0		1.0		0.0		0.0		
2.0		1.73		1.0		1.0		1.73		
3.0		1.5		-1.5		2.6		2.6		
4.0		0.0		-4.0		4.0		0.0		
5.0		-2.5		-2.5		4.33		-4.33		
6.0		-5.2		3.0		3.0		-5.2		

Here are the summations of the columns of the table, curated for you :
Y       =  21.0
Ycosx   =  -3.4641016151377526
Ycos2x  =  -3.000000000000001
Ycos3x  =  14.92820323027551
Ysinx   =  14.92820323027551
Ysin2x  =  14.92820323027551
Ysin3x  =  -5.196152422706631


In [3]:
help(print_summations)

Help on function print_summations in module __main__:

print_summations(table)
    Prints the summations of the columns in the given table.
    
    Args:
        table: The table to print the summations for.

